### Semantic Kernel Agent using MCP Tools via Azure API Management

Model Context Protocol with Azure API Management to enable plug & play of tools for a Semantic Kernel Agent

0️⃣ Initialize Modules

In [1]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPSsePlugin
from dotenv import load_dotenv
import os

load_dotenv(override=True)

apim_resource_gateway_url=os.getenv("APIM_RESOURCE_GATEWAY_URL")
apim_subscription_key=os.getenv("APIM_SUBSCRIPTION_KEY")
openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION")               
openai_deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

1️⃣ Create the Semantic Kernel Orchestrator

In [2]:
async def main(user_input: str):
    # 1. Create the agent
    async with MCPSsePlugin(
        name="AgentPlugin",
        url=f"{apim_resource_gateway_url}/agent/sse",
        description="Menu Agent Plugin",
    ) as agent_plugin, \
    MCPSsePlugin(
        name="AnotherPlugin",
        url=f"{apim_resource_gateway_url}/weather/sse",
        description="Weather Plugin",
    ) as weather_plugin:
              
        orchestrator = ChatCompletionAgent(
            service=AzureChatCompletion(
                endpoint=apim_resource_gateway_url,
                api_key=apim_subscription_key,
                api_version=openai_api_version,                
                deployment_name=openai_deployment_name
            ),
            name="Orchestrator",
            instructions="Route the request to the appropriate agent.",
            plugins=[agent_plugin, weather_plugin],
        )

        thread: ChatHistoryAgentThread | None = None

        print(f"# User: {user_input}")
        # 2. Invoke the agent for a response
        response = await orchestrator.get_response(messages=user_input, thread=thread)
        print(f"# {response.name}: {response} ")
        thread = response.thread  # type: ignore

        # 3. Cleanup: Clear the thread
        await thread.delete() if thread else None


In [3]:
user_input = "What are special itens in the menu?"
await main(user_input)

# User: What are special itens in the menu?
# Orchestrator: The special items on the menu are:

- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Let me know if you'd like more details or prices for any of these specials! 


In [4]:
user_input = "What is the weather in lisbon?"
await main(user_input)

# User: What is the weather in lisbon?
# Orchestrator: The weather in Lisbon is currently cloudy, with a temperature of about 1.9°C and humidity at 92.7%. If you need a more detailed forecast or specific date, just let me know! 


In [5]:
user_input = "What are special itens in the menu? What is the weather in lisbon?"
await main(user_input)

# User: What are special itens in the menu? What is the weather in lisbon?
# Orchestrator: Here are the answers to your questions:

Special items on the menu:
- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Current weather in Lisbon:
- Condition: Windy
- Temperature: -0.83°C
- Humidity: 34.79%

Let me know if you want more details about the menu specials or anything else! 
